# Sparse Subspace Clustering (Vidal IEEE Trans. on PAMI 2013)

**References:**


1.   Main Paper (Vidal IEEE Trans. on PAMI 2013): https://arxiv.org/pdf/1203.1005.pdf
2.   Supplementary Paper (Vidal CVPR 2009): http://cis.jhu.edu/~ehsan/Downloads/SSC-CVPR09-Ehsan.pdf
3.   Spectral Clustering: http://people.csail.mit.edu/dsontag/courses/ml14/notes/Luxburg07_tutorial_spectral_clustering.pdf
4.   Spectral Clustering Code: https://juanitorduz.github.io/spectral_clustering/











Our objective is given a set of points drawn from a `union of subspaces`, we need to find the number of subspaces, their dimensions, a basis for each subspace, and the segmentation of the data.

Let all the points be from $D$ dimensional space, here, for our use, we consider the value of $D$ to be $10$. Let us consider that we have $N=100$ $D$ dimensional data points.

In [1]:
import numpy as np
import cvxpy as cp
from cvxpy.atoms.elementwise.power import power

##### Set Parameters of the data space

In [2]:
N = 150 # Total no of points
D = 4  # Dimension of space
K = 3 # Number of clusters

In [3]:
from sklearn.datasets import load_iris
iris_data = load_iris()
iris_X = np.asarray(iris_data.data)
input_data = iris_X.T

The input_data is shaped in the form of $(4,150)$ i.e. in the form of $(D,N)$ where each column represents each $D$ dimensional data point, total number of such data points being $N$.

In [7]:
def find_sparse_sol(Y,i,N):
  if i == 0:
    Ybari = Y[:,1:N]
  if i == N-1:
    Ybari = Y[:,0:N-1]
  if i!=0 and i!=N-1:
    Ybari = np.concatenate((Y[:,0:i],Y[:,i+1:N]),axis=1)
  yi = Y[:,i].reshape(D,1)
  # this ci will contain the solution of the l1 optimisation problem  min |ci|1   yi = Ybari*ci
  ci = cp.Variable(shape=(N-1,1))
  constraint = [cp.sum(ci)==1] #Ybari * ci == yi, 
  obj = cp.Minimize(power(cp.norm(yi-Ybari@ci,2),2) + 0.082*cp.norm(ci,1))
  prob = cp.Problem(obj, constraint)
  prob.solve()
  return ci.value

The above function `find_sparse_sol` finds the solution to the following optimisation problem $(\textrm{min} \; ||y_i - Y_{\hat{i}}c_i||_{F}^2 + \lambda||c_i||_1)$ subject to the constraint $\sum_{i=i}^{N}c_i=1$. We use this optimisation term to *deal with all the possible noise values in the data.*

In [9]:
C = np.concatenate((np.zeros((1,1)),find_sparse_sol(input_data,0,N)),axis=0)

for i in range(1,N):
  ci = find_sparse_sol(input_data,i,N)
  zero_element = np.zeros((1,1))
  cif = np.concatenate((ci[0:i,:],zero_element,ci[i:N,:]),axis=0)
  C = np.concatenate((C,cif),axis=1)
print(C.shape)

/home/heisenberg/Desktop/K-means/kmeans/lib/python3.8/site-packages/cvxpy/problems/problem.py:1054: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


(150, 150)


We now include a zero vector of size $(1,1)$ at the $i$ th position of $c_i$ to form $\hat{c_i}$ which we represent as *cif* in the code.
Then we concatenate $[\hat{c_1}$, $\hat{c_2}$, ..., $\hat{c_N}]$ to form the matrix $C$.  

$C$ is the *Matrix of Coefficients* and is of the form $C = [\hat{c_1}, \hat{c_2}, ..., \hat{c_N}] \in \mathbb{R}^{NXN}$.

In [10]:
Cp = np.zeros((C.shape[0], C.shape[1]))
for i in range(N):
  for j in range(N):
    Cp[i,j] = abs(C[i,j]) + abs(C[j,i])

print(Cp.shape)

(150, 150)


In [11]:
cz = 0
for i in range(Cp.shape[0]):
    for j in range(Cp.shape[1]):
        if Cp[i,j] < 1e-5:
            cz += 1
print(cz)

1644


In the above code block, we make $C$ a symmetric matrix by the operation $C_{ij} = |C_{ij}| + |C_{ji}|$. It is still a valid representation of the similarity since if $y_i$ can write itself as a linear combination of all the points in the same subspace including $y_j$, then $y_j$ can also be represented as a linear combination of all the other points belonging to the same subspace including $c_i$.
            In the above code block, we print the number of $0$'s in the matrix $Cp$, we denote any value less then $e^{-5}$ as $0$.

In [12]:
D = np.zeros((N,N))
sum_list=[]
for i in range(N):
  csum = np.sum(Cp[:,i],axis=0)
  sum_list.append(csum)

D = np.diag(sum_list)
print(D)

[[1.24708066 0.         0.         ... 0.         0.         0.        ]
 [0.         1.38384376 0.         ... 0.         0.         0.        ]
 [0.         0.         1.26239179 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.21430676 0.         0.        ]
 [0.         0.         0.         ... 0.         1.98337482 0.        ]
 [0.         0.         0.         ... 0.         0.         1.27050197]]


$D$ is the `degree matrix`. In this case, $D \in \mathbb{R}^{NxN}$ is a diagonal matrix with $D_{ii} = \sum_{j}C_{ij}$. 

In [13]:
L = np.subtract(D, Cp)
print(L.shape)

(150, 150)


This $L$ is the Laplacian matrix, which can be defined as $L = D - C$. Next, we calculate the `eigenvalues` and `eigenvectors` of the Laplacian matrix, which we will use for Spectral clustering of the data points. $L$ is a *positive, semi-definite matrix* this means all the eigenvalues of the matrix will be greater than equal to $0$.

### Perform Spectral Clustering with Laplacian Matrix L

In [14]:
from scipy import linalg

eigenvals, eigenvcts = linalg.eig(L)

eigenvals = np.real(eigenvals)
eigenvcts = np.real(eigenvcts)

eig = eigenvals.reshape((N,1))

Sort Eigen Values

In [15]:
eigenvals_sorted_indices = np.argsort(eigenvals)
eigenvals_sorted = eigenvals[eigenvals_sorted_indices]

In [16]:
indices = []
for i in range(0,K):
    ind = []
    print(eigenvals_sorted_indices[i])
    ind.append(eigenvals_sorted_indices[i])
    indices.append(np.asarray(ind))

24
25
36


In the above code, we find out the indices of the eigenvectors corresponding to the $K$ smallest eigenvectors.

In [17]:
indices = np.asarray(indices)

In [18]:
zero_eigenvals_index = np.array(indices)

Here, the indices are arranged according to their sorted order of values and the sorted eigen values are stored in *eigenvals_sorted*.

In [19]:
eigenvals[zero_eigenvals_index]

array([[8.26898479e-16],
       [3.81158116e-01],
       [6.60406524e-01]])

In [20]:
import pandas as pd

proj_df = pd.DataFrame(eigenvcts[:, zero_eigenvals_index.squeeze()])
proj_df.columns = ['v_' + str(c) for c in proj_df.columns]
proj_df.head()

,v_0,v_1,v_2
0,0.08165,-0.124575,0.034847
1,0.08165,-0.108455,0.082008
2,0.08165,-0.130572,0.004090
3,0.08165,-0.132622,0.032966
4,0.08165,-0.124779,-0.040015


Stack the Eigen Vectors corresponding to the zero Eigen Values in a dataframe *proj_df*. This can be thought of as a $N X K$ matrix where the columns denote an eigen vector and the rows denote a data point.

Apply *K-Means Clustering* with $K = 3$.

In [21]:
from sklearn.cluster import KMeans

def run_k_means(df, n_clusters):
    k_means = KMeans(random_state=25, n_clusters=n_clusters)
    k_means.fit(df)
    cluster = k_means.predict(df)
    return cluster

cluster = run_k_means(proj_df, n_clusters=K)

*run_k_means* applies `K-Means Clustering` on *proj_df* with number of clusters = $3$.The clustering of the data points is returned in the variable *'cluster'*.

Display clusters formed

In [22]:
print(cluster)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 0 2 2 2 0 0 2 2 0 2 0 2 2 0 2 0 2 2 2 0 2 2 2
 2 2 2 2 0 2 2 2 2 0 0 0 2 2 0 2 2 0 2 2 2 0 0 2 0 2 0 0 0 0 0 2 0 2 2 0 0
 0 0 0 0 0 0 0 2 2 0 0 2 2 0 2 0 0 0 2 2 0 0 2 2 2 0 0 0 0 0 0 0 0 0 0 2 0
 0 0]


As we can see, the data points have been clustered into two subspaces: $0,1 \, \textrm{and} \, 2$ corresponding to the $3$ subspaces that we have considered. 

In [23]:
c0 = 0
for l in cluster:
  if l == 0:
    c0 += 1
print(c0)

53


In [24]:
c1 = 0
for l in cluster:
  if l == 1:
    c1 += 1
print(c1)

50


In [25]:
c2 = 0
for l in cluster:
  if l == 2:
    c2 += 1
print(c2)

47


$53$ data points have been labelled to the 1st subspace, $50$ data points have been labelled to the 2nd subspace and $47$ data points have been labelled to the 3rd subspace.

In [26]:
orig = iris_data.target

In [27]:
pred = np.asarray(cluster)

In [28]:
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import normalized_mutual_info_score
print("ARI = " + str(adjusted_rand_score(orig, pred)))
print("NMI = " + str(normalized_mutual_info_score(orig, pred)))

ARI = 0.5666502143559329
NMI = 0.6248401785062485


In the above code block, we calculate the `Adjusted random score` and the `normalized mutual info score` between the `original` and the `predicted` labels for the various data points.